In [162]:
import pandas as pd
from pathlib import Path
from os import listdir
import os.path
from collections import defaultdict
import re
import numpy as np
pd.set_option('display.float_format', lambda x: '%.5f' % x)


In [163]:
repo_dir = Path("/home/luk/repos/RAPid-Learn/data")
experiment_id = "2267a0875d2e49f5b1f910d9b0f7df0b-2021-09-30_22:10:50-PPO-10000episodes-rewardshapingon"
experiment_path = repo_dir / experiment_id

In [164]:
novelties = ["axefirecteasy", "prenovelty", "axetobreakeasy", "axetobreakhard", "firecraftingtableeasy",
             "firecraftingtablehard", "rubbertree", "rubbertreehard", "scrapeplank"]
CONVERGENCE_SUCCESS_RATIO = 0.96

In [165]:
def extract_number(f):
    s = re.findall("\d+", f)
    return int(s[0]) if s else -1

In [166]:
csvs_per_novelty = defaultdict(list)
for novelty in novelties:
    for trial_dir in listdir(experiment_path / novelty):
        if os.path.isdir(experiment_path / novelty / trial_dir):
            for filename in os.listdir(experiment_path / novelty / trial_dir):
                if filename.endswith(".csv"):
                    csvs_per_novelty[novelty].append(experiment_path / novelty / trial_dir / filename)
                                       

In [167]:
dfs = []
timesteps_dfs = []
for novelty in novelties:

    for csv_path in csvs_per_novelty[novelty]:
        
        df = pd.read_csv(csv_path, skiprows=[0])

        df["trial"] = extract_number(str(os.path.basename(os.path.normpath(csv_path))))
        df["novelty"] = novelty
        
        df["ts_trained"] = df["l"].loc[df['mode'] == "learn"].cumsum()
        df["ts_trained"].ffill(inplace=True)
        if novelty == "prenovelty":
            df["episode_counter"] = df.index
            df["episode_counter"].loc[df['mode'] == "eval"] = np.nan
            df["episode_counter"].ffill(inplace=True)
            
        df["trial_converged"] = False
        for trial_id in list(df["trial"].unique()):
            trial_section = df.loc[(df['trial'] == trial_id) & (df['mode'] == "eval")]
            was_successful_df = trial_section.groupby("episode_counter").agg({'success': ['mean'], 'ts_trained': ['mean']}).reset_index()
            was_successful_df["trial"] = trial_id
            was_successful_df["novelty"] = novelty
            timesteps_dfs.append(was_successful_df)
            
        dfs.append(df)
        
results_df = pd.concat(dfs).reset_index()
timesteps_df = pd.concat(timesteps_dfs).reset_index()

/home/luk/programs/miniconda3/envs/rapid-learn/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [168]:
agg_results_df = results_df.loc[results_df['mode'] == "eval"] \
.groupby(by=["novelty", "episode_counter"]) \
.agg({'success': ['mean', 'std'], 'r': ['mean', 'std']}).reset_index()

In [169]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(agg_results_df)

                   novelty episode_counter success                  r  \
                                              mean     std       mean   
0            axefirecteasy       102.00000 0.00000 0.00000 -293.00000   
1            axefirecteasy       203.00000 0.00000 0.00000 -163.00000   
2            axefirecteasy       304.00000 0.00000 0.00000 -106.00000   
3            axefirecteasy       405.00000 0.00000 0.00000 -105.00000   
4            axefirecteasy       506.00000 0.00000 0.00000 -107.00000   
5            axefirecteasy       607.00000 0.00000 0.00000 -103.00000   
6            axefirecteasy       708.00000 0.00000 0.00000 -102.00000   
7            axefirecteasy       809.00000 0.00000 0.00000 -106.00000   
8            axefirecteasy       910.00000 0.00000 0.00000 -102.00000   
9            axefirecteasy      1011.00000 0.00000 0.00000 -107.00000   
10           axefirecteasy      1112.00000 0.00000 0.00000 -100.00000   
11           axefirecteasy      1213.00000 0.00000 

In [170]:
results_df.to_csv(repo_dir / experiment_id / "full_results.csv")

In [171]:
agg_results_df.to_csv(repo_dir / experiment_id / "parsed_results.csv")

In [172]:
timesteps_df_successful = timesteps_df.loc[timesteps_df['success']["mean"] > CONVERGENCE_SUCCESS_RATIO]
timesteps_df_successful.columns = [' '.join(col).strip() for col in timesteps_df_successful.columns.values]

In [173]:
timesteps_df_successful

,index,episode_counter,success mean,ts_trained mean,trial,novelty
501,0,9999.00000,1.00000,515505.00000,1,prenovelty
502,0,9999.00000,0.99000,1029339.00000,2,prenovelty
510,5,607.00000,1.00000,88675.00000,4,axetobreakeasy
511,6,708.00000,1.00000,95209.00000,4,axetobreakeasy
512,7,809.00000,1.00000,102959.00000,4,axetobreakeasy
...,...,...,...,...,...,...
4000,95,9697.00000,1.00000,593691.00000,5,scrapeplank
4001,96,9798.00000,1.00000,595679.00000,5,scrapeplank
4002,97,9899.00000,1.00000,597680.00000,5,scrapeplank
4003,98,10000.00000,1.00000,599618.00000,5,scrapeplank


In [178]:
timesteps_df_grouped = timesteps_df_successful.loc[timesteps_df_successful.groupby(["novelty", "trial"])["ts_trained mean"].idxmin()].groupby("novelty").agg({'ts_trained mean': ['mean', 'std']})
timesteps_df_grouped

ts_trained mean             
                                 mean          std
novelty                                           
axetobreakeasy            63308.40000  23304.06024
axetobreakhard           366131.00000 237679.25197
firecraftingtableeasy    488578.25000 118876.34174
firecraftingtablehard   1002070.00000          NaN
prenovelty               772422.00000 363335.50580
rubbertree               908827.00000          NaN
scrapeplank              405550.75000 126768.21894